In [6]:
using Pkg
using Revise
Pkg.activate(dirname(dirname(@__DIR__)))
using MacroEnergy
using Gurobi
using Plots
using DataFrames, CSV
using BenchmarkTools
using JLD2, FileIO
using JuMP

  Activating project at `~/Documents/MacroEnergy.jl`


In [15]:
system = MacroEnergy.load_system(@__DIR__)
#co2_node = MacroEnergy.get_nodes_sametype(system.locations, CO2)[1]

┌ Info: Loading system from /Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_cement_co2_cap/system_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system.jl:18
┌ Info: Loading system data
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:18
┌ Info: Done loading system data. It took 0.0 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_system_data.jl:26
┌ Info: Generating system
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/generate_system.jl:18
┌ Warning: WeightTotal not found in time_data.json
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:81
┌ Info: Using PeriodLength as default value for WeightTotal
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/load_inputs/load_time_data.jl:82
┌ Info: Done generating system. It took 0.2 seconds
└ @ MacroEnergy /Users/al3792/Documents/Macro

MacroEnergy.System("/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_cement_co2_cap", (ConstraintScaling = true, WriteSubcommodities = true), Dict{Symbol, DataType}(:CO2Captured => CO2Captured, :Electricity => Electricity, :CO2 => CO2, :Cement => Cement, :NaturalGas => NaturalGas, :CementMaterials => CementMaterials), Dict{Symbol, MacroEnergy.TimeData}(:Cement => MacroEnergy.TimeData{Cement}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :NaturalGas => MacroEnergy.TimeData{NaturalGas}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :CO2 => MacroEnergy.TimeData{CO2}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :Electricity => MacroEnergy.TimeData{Electricity}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(1 => 1.0), Dict(1 => 1)), :CementMaterials => MacroEnergy.TimeData{CementMaterials}(1:1:8760, 1, StepRange{Int64, Int64}[1:1:8760], [1], Dict(

In [16]:
model = MacroEnergy.generate_model(system)

┌ Info: Generating model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:3
┌ Info:  -- Adding linking variables
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:15
┌ Info:  -- Defining available capacity
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:18
┌ Info:  -- Generating planning model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:21
┌ Info: Retrofit_ids: Any[1]
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/model/constraints/constraints_utils.jl:23
┌ Info:  -- Generating operational model
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:24
┌ Info:  -- Model generation complete. It took 7.45 seconds
└ @ MacroEnergy /Users/al3792/Documents/MacroEnergy.jl/src/generate_model.jl:29


A JuMP Model
├ solver: none
├ objective_sense: MIN_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 604487
├ num_constraints: 1454235
│ ├ AffExpr in MOI.EqualTo{Float64}: 297842
│ ├ AffExpr in MOI.GreaterThan{Float64}: 78840
│ ├ AffExpr in MOI.LessThan{Float64}: 490587
│ ├ VariableRef in MOI.EqualTo{Float64}: 16
│ └ VariableRef in MOI.GreaterThan{Float64}: 586950
└ Names registered in the model
  └ :cRetrofitCapacity, :eFixedCost, :eRetrofitCapByRetroId, :eRetrofittedCapByRetroId, :eVariableCost, :vREF

In [17]:
MacroEnergy.set_optimizer(model, Gurobi.Optimizer);
MacroEnergy.set_optimizer_attributes(model, "BarConvTol"=>1e-3,"Crossover" => 0, "Method" => 2)
MacroEnergy.optimize!(model)

Set parameter Username
Set parameter LicenseID to value 2637694
Academic license - for non-commercial use only - expires 2026-03-17
Set parameter BarConvTol to value 0.001
Set parameter Crossover to value 0
Set parameter Method to value 2
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 0.001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 24.2.0 24C101)

CPU model: Apple M4 Pro
Thread count: 14 physical cores, 14 logical processors, using up to 14 threads

Non-default parameters:
Method  2
BarConvTol  0.001
Crossover  0

Optimize a model with 867269 rows, 604487 columns and 2924804 nonzeros
Model fingerprint: 0x1fcc5b91
Coefficient statistics:
  Matrix range     [4e-07, 2e+04]
  Objective range  [1e-01, 2e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 1e+06]
Presolve removed 421043 rows and 219559 columns
Presolve time: 1.04s
Presolved: 446226 rows, 384928 columns, 2100798 nonzeros
Ordering ti

In [18]:
include("../helper_functions.jl")

get_co2_node_values (generic function with 1 method)

In [19]:
get_system_results(system)

Row,objective_value,cement_co2_sink,co2_sink,co2_captured_sink
,Float64,Float64,Float64,Float64
1,1.70816e9,1.19616e6,2.16889e7,0.0


In [20]:
case = "20pct_co2_reduction"
results_dir = joinpath(@__DIR__,  "results")
mkpath(results_dir)
save_results(results_dir, case, system)

"/Users/al3792/Documents/MacroEnergy.jl/ExampleSystems/cement_retrofit_cement_co2_cap/results/20pct_co2_reduction_flows.csv"

In [4]:
model[:cRetrofitCapacity]

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, Any[1]
And data, a 1-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 cRetrofitCapacity[1] : 114.15 vRETROFITUNIT_cement_MA_cement_edge + 114.15 vRETROFITUNIT_cement_CT_cement_edge + 114.15 vRETROFITUNIT_cement_ME_cement_edge - 114.15 vNEWUNIT_cement_MA_echem_cement_edge - 114.15 vNEWUNIT_cement_CT_echem_cement_edge - 114.15 vNEWUNIT_cement_ME_echem_cement_edge = 0

In [19]:
system.assets[1].cement_edge.constraints

1-element Vector{AbstractTypeConstraint}:
 CapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -114.15 vNEWUNIT_cement_MA_cement_edge + 114.15 vRETUNIT_cement_MA_cement_edge + 114.15 vRETROFITUNIT_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[1] ≤ 0
 -114.15 vNEWUNIT_cement_MA_cement_edge + 114.15 vRETUNIT_cement_MA_cement_edge + 114.15 vRETROFITUNIT_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[2] ≤ 0
 -114.15 vNEWUNIT_cement_MA_cement_edge + 114.15 vRETUNIT_cement_MA_cement_edge + 114.15 vRETROFITUNIT_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[3] ≤ 0
 -114.15 vNEWUNIT

In [20]:
system.assets[4].cement_edge.constraints

1-element Vector{AbstractTypeConstraint}:
 CapacityConstraint(missing, missing, 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:1:8760
And data, a 8760-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 -114.15 vNEWUNIT_cement_MA_cement_edge + 114.15 vRETUNIT_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[1] ≤ 0
 -114.15 vNEWUNIT_cement_MA_cement_edge + 114.15 vRETUNIT_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[2] ≤ 0
 -114.15 vNEWUNIT_cement_MA_cement_edge + 114.15 vRETUNIT_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[3] ≤ 0
 -114.15 vNEWUNIT_cement_MA_cement_edge + 114.15 vRETUNIT_cement_MA_cement_edge + vFLOW_cement_MA_cement_edge[4] ≤ 0
 -114.15 vNEWUNIT_cement_MA_cement_